solutions:

propagation asyn.: cnt->concat(features,ngh structure)

conservation: normalize as the tot in local/global

propagation dec.: dis&time->dec function

local structure feature(sample+global vec)

kernel methods

# asynchronous propagation
def asynchronous_prop():

# local strucuture sample
def local_structure_sample():

# propagation descent
def propgation_heat_descent():

# k-hop interaction
def k_hop_interaction():

### 2 layers

In [1]:
from __future__ import division
from __future__ import print_function

In [2]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import torch
from scipy.sparse import csgraph
import scipy.sparse.linalg
import sys
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [3]:
def preprocess_HK(A, alpha: float, eps: float):
    N = A.shape[0]

    # Self-loops
    A_loop = sp.eye(N) + A

    # Symmetric transition matrix
    D_loop_vec = A_loop.sum(0).A1
    D_loop_vec_invsqrt = 1 / np.sqrt(D_loop_vec)
    D_loop_vec_invsqrt[np.isinf(D_loop_vec_invsqrt)] = 0.
    D_loop_invsqrt = sp.diags(D_loop_vec_invsqrt)
    T_sym = A.dot(D_loop_invsqrt).transpose().dot(D_loop_invsqrt).tocoo()#D_loop_invsqrt @ A_loop @ D_loop_invsqrt

    # PPR-based diffusion
    S = alpha * sp.linalg.inv(sp.eye(N) - (1 - alpha) * T_sym)

    # Sparsify using threshold epsilon
    S_tilde = S.multiply(S >= eps)

    # Column-normalized transition matrix on graph S_tilde
    D_tilde_vec = S_tilde.sum(0).A1
    T_S = S_tilde / D_tilde_vec

    T_S = scipy.sparse.csr_matrix(T_S)

    print("T_S.shape",T_S.shape)

    print("T_S.dtype",T_S.dtype)

    print("T_S.type",type(T_S))
    
    return T_S

In [4]:
def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def normalize_adj(mx):#D^(-1/2)^T*A*D^(-1/2) preprocess 得到一个领域传播位权矩阵
    """Row-normalize sparse matrix"""
    """
    normalize_adj process
    adj--original matrix shape (2708, 2708)
    adj--degree sum matrix shape (2708, 1)
    adj--degree inv matrix shape (2708,)
    adj--degree no inf matrix shape (2708,)
    adj--degree norm diag matrix shape (2708, 2708)
    adj--res shape (2708, 2708)
    """
    ## print("normalize_adj process")
    ## print("adj--original matrix shape",mx.shape)
    rowsum = np.array(mx.sum(1)) #每行求和，求得每个节点的出度列矩阵
    ## print("adj--degree sum matrix shape",rowsum.shape)
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    ## print("adj--degree inv matrix shape",r_inv_sqrt.shape)
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    ## print("adj--degree no inf matrix shape",r_inv_sqrt.shape)
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    ## print("adj--degree norm diag matrix shape",r_mat_inv_sqrt.shape)
    resm = mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo()
    ## print("adj--res shape",resm.shape)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo() #D^(-1/2)*A*D^(-1/2)^T

def normalize(mx):#D^-(1)*A
    """Row-normalize sparse matrix"""
    """
    normalize process
    mx--original matrix shape (2708, 1433)
    mx--degree sum matrix shape (2708, 1)
    mx--degree inv matrix shape (2708,)
    mx--degree no inf matrix shape (2708,)
    mx--degree norm diag matrix shape (2708, 2708)
    mx--res shape (2708, 1433)
    """  
    ## print("normalize process")
    ## print("mx--original matrix shape",mx.shape)
    rowsum = np.array(mx.sum(1))
    ## print("mx--degree sum matrix shape",rowsum.shape)
    r_inv = np.power(rowsum, -1).flatten()
    ## print("mx--degree inv matrix shape",r_inv.shape)
    r_inv[np.isinf(r_inv)] = 0.
    ## print("mx--degree no inf matrix shape",r_inv.shape)
    r_mat_inv = sp.diags(r_inv)
    ## print("mx--degree norm diag matrix shape",r_mat_inv.shape)
    mx = r_mat_inv.dot(mx)
    ## print("mx--res shape",mx.shape)

    print("NA.shape",mx.shape)

    print("NA.dtype",mx.dtype)

    print("NA.type",type(mx))
    return mx

def laplacian(mx, norm):
    """Laplacian-normalize sparse matrix"""
    assert (all (len(row) == len(mx) for row in mx)), "Input should be a square matrix"

    return csgraph.laplacian(adj, normed = norm)

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def load_data(path="../Data", dataset="cora"):
    """
    ind.[:dataset].x     => the feature vectors of the training instances (scipy.sparse.csr.csr_matrix)
    ind.[:dataset].y     => the one-hot labels of the labeled training instances (numpy.ndarray)
    ind.[:dataset].allx  => the feature vectors of both labeled and unlabeled training instances (csr_matrix)
    ind.[:dataset].ally  => the labels for instances in ind.dataset_str.allx (numpy.ndarray)
    ind.[:dataset].graph => the dict in the format {index: [index of neighbor nodes]} (collections.defaultdict)
    ind.[:dataset].tx => the feature vectors of the test instances (scipy.sparse.csr.csr_matrix)
    ind.[:dataset].ty => the one-hot labels of the test instances (numpy.ndarray)
    ind.[:dataset].test.index => indices of test instances in graph, for the inductive setting
    """
    print("\n[STEP 1]: Upload {} dataset.".format(dataset))

    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []

    for i in range(len(names)):
        with open("{}/ind.{}.{}".format(path, dataset, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    
    ## print("x.shape=",x.shape)
    ## print("y.shape=",y.shape)
    ## print("tx.shape=",tx.shape)
    ## print("ty.shape=",ty.shape)
    ## print("allx=",allx.shape)
    ## print("ally=",ally.shape)
    ## print("len(graph)=",len(graph))
    """
    x.shape= (140, 1433)
    y.shape= (140, 7)
    tx.shape= (1000, 1433)
    ty.shape= (1000, 7)
    allx= (1708, 1433)
    ally= (1708, 7)
    len(graph)= 2708
    """
    
    ## print("x=",x)
    ## print("y=",y)
    ## print("tx=",tx)
    ## print("ty=",ty)
    ## print("allx=",allx)
    ## print("ally=",ally)
    ## print("graph=",graph)

    test_idx_reorder = parse_index_file("{}/ind.{}.test.index".format(path, dataset))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset == 'citeseer':
        #Citeseer dataset contains some isolated nodes in the graph
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended

        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]

    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
    print("| # of nodes : {}".format(adj.shape[0]))
    print("| # of edges : {}".format(adj.sum().sum()/2))

    features = normalize(features) #将feature按照度归一化
    adj = normalize_adj(adj + sp.eye(adj.shape[0])) #将A按照两个边端点度归一化 PPR方法
    #adj = preprocess_HK(adj+ sp.eye(adj.shape[0]),0.5,0.0001)
    
    ###### 
    ### AF MATRIX
    ######
    #####################
    ########################## adj = AF_matrix(adj, features, 3)
    #####################
    ## AF = AF_matrix(adj, features, 3)
    
    print("| # of features : {}".format(features.shape[1]))
    print("| # of clases   : {}".format(ally.shape[1]))

    features = torch.FloatTensor(np.array(features.todense()))
    sparse_mx = adj.tocoo().astype(np.float32)
    adj = torch.FloatTensor(np.array(adj.todense()))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    if dataset == 'citeseer':
        save_label = np.where(labels)[1]
    labels = torch.LongTensor(np.where(labels)[1])

    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)
    idx_test = test_idx_range.tolist()

    print("| # of train set : {}".format(len(idx_train)))
    print("| # of val set   : {}".format(len(idx_val)))
    print("| # of test set  : {}".format(len(idx_test)))

    idx_train, idx_val, idx_test = list(map(lambda x: torch.LongTensor(x), [idx_train, idx_val, idx_test]))

    def missing_elements(L):
        start, end = L[0], L[-1]
        return sorted(set(range(start, end+1)).difference(L))

    if dataset == 'citeseer':
        L = np.sort(idx_test)
        missing = missing_elements(L)

        for element in missing:
            save_label = np.insert(save_label, element, 0)

        labels = torch.LongTensor(save_label)

    return adj, features, labels, idx_train, idx_val, idx_test

In [5]:
def normalize_adj_torch(torch_mx):#D^(-1/2)^T*A*D^(-1/2) preprocess 得到一个领域传播位权矩阵
    """Row-normalize sparse matrix"""
    """
    normalize_adj process
    adj--original matrix shape (2708, 2708)
    adj--degree sum matrix shape (2708, 1)
    adj--degree inv matrix shape (2708,)
    adj--degree no inf matrix shape (2708,)
    adj--degree norm diag matrix shape (2708, 2708)
    adj--res shape (2708, 2708)
    """
    ## print("normalize_adj process")
    ## print("adj--original matrix shape",mx.shape)
    """
    rowsum = np.array(torch_mx.sum(1)) #每行求和，求得每个节点的出度列矩阵
    ## print("adj--degree sum matrix shape",rowsum.shape)
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    ## print("adj--degree inv matrix shape",r_inv_sqrt.shape)
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    ## print("adj--degree no inf matrix shape",r_inv_sqrt.shape)
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    ## print("adj--degree norm diag matrix shape",r_mat_inv_sqrt.shape)
    resm = np.array(torch_mx).dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo()
    ## print("adj--res shape",resm.shape)
    resm = torch.from_numpy(resm)
    """
    rowsum = torch_mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum,-0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)]=0.
    r_mat_inv_sqrt = torch.diag_embed(r_inv_sqrt)
    resm = torch.mm(r_mat_inv_sqrt.T,torch_mx)
    resm = torch.mm(resm,r_mat_inv_sqrt)
    ## resm.requires_grad = False only can change on the leaf node
    return resm#mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo() #D^(-1/2)*A*D^(-1/2)^T

In [6]:
def normalize_adj_torch_khop(torch_mx,hop_num,alpha):#D^(-1/2)^T*A*D^(-1/2) preprocess 得到一个领域传播位权矩阵
    """Row-normalize sparse matrix"""
    """
    normalize_adj process
    adj--original matrix shape (2708, 2708)
    adj--degree sum matrix shape (2708, 1)
    adj--degree inv matrix shape (2708,)
    adj--degree no inf matrix shape (2708,)
    adj--degree norm diag matrix shape (2708, 2708)
    adj--res shape (2708, 2708)
    """
    ## print("normalize_adj process")
    ## print("adj--original matrix shape",mx.shape)
    """
    rowsum = np.array(torch_mx.sum(1)) #每行求和，求得每个节点的出度列矩阵
    ## print("adj--degree sum matrix shape",rowsum.shape)
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    ## print("adj--degree inv matrix shape",r_inv_sqrt.shape)
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    ## print("adj--degree no inf matrix shape",r_inv_sqrt.shape)
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    ## print("adj--degree norm diag matrix shape",r_mat_inv_sqrt.shape)
    resm = np.array(torch_mx).dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo()
    ## print("adj--res shape",resm.shape)
    resm = torch.from_numpy(resm)
    """
    for i in range(0,hop_num):
        rowsum = torch_mx.sum(1)
        r_inv_sqrt = torch.pow(rowsum,-0.5).flatten()
        r_inv_sqrt[torch.isinf(r_inv_sqrt)]=0.
        r_mat_inv_sqrt = torch.diag_embed(r_inv_sqrt)
        resm_new = torch.mm(r_mat_inv_sqrt.T,torch_mx)
        resm_new = torch.mm(resm_new,r_mat_inv_sqrt)
        if(i == 0):
            resm = resm_new
        else:
            resm = resm + resm_new * alpha
            alpha *= alpha
        torch_mx = torch.mm(torch_mx,torch_mx)
    ## resm.requires_grad = False only can change on the leaf node
    return resm#mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo() #D^(-1/2)*A*D^(-1/2)^T

In [7]:
def normalize_adj_torch_interaction_conservation(adj):
    linesum = adj.sum(1)
    line_degree = linesum.expand(adj.shape[1],adj.shape[1])
    adj = adj/line_degree

    return adj

In [8]:
def normalize_diffusion_descent_multi_times(torch_mx,hop_num,alpha):#D^(-1/2)^T*A*D^(-1/2) preprocess 得到一个领域传播位权矩阵
    
    for i in range(0,hop_num):
        normalize_adj_torch_interaction_conservation(torch_mx)
        # i-->hop_num
        # resm_new(i,j)*alpha means the similarity betweent the i and j, it can be view as the 1./v
        # as for the diffusion formula, that is the f = C*exp(-d*v) the larger distance and larger speed 
        # can result in a great diffusion factor while the little distance with small speed can result in 
        # a slow speed.
        # Here another is the transferring time, here we consider a small domain quick transferring scenery
        # where the transferring speed is constant and the interaction happens quickly where
        # the temperature will have no change, the asynchronous update will show as multiple time quick update
        rowsum = torch_mx.sum(1)
        r_inv_sqrt = torch.pow(rowsum,-0.5).flatten()
        r_inv_sqrt[torch.isinf(r_inv_sqrt)]=0.
        r_mat_inv_sqrt = torch.diag_embed(r_inv_sqrt)
        resm_new = torch.mm(r_mat_inv_sqrt.T,torch_mx)
        resm_new = torch.mm(resm_new,r_mat_inv_sqrt) # here resm_new represents the v matrix, larger v, more correlated
        ### here means the resm_new as v
        ### factor_m = torch.exp(-(i+1)*resm_new) # here factor_m represents the diffusion decreases matri
        ### factor_m[factor_m==1] = 0 #here must be the similarity be 0
        
        # Descent_Factor = C*exp(-d^2/t) = C*exp(-d*v) 
         
        
        # hop_num

        ### here means the resm_new as 1/v
        expmm = (i+1)/resm_new
        expmm[torch.isinf(expmm)] = 0
        factor_m = torch.exp(-expmm)
        # print("prefactorsm.shape",factor_m.shape)
        # print("prefactorm",factor_m)

        ### here means the resm_new as v
        ### times_m = (i+1)/resm_new # here the d./v matix shows the transferring times matrix
        ### times_m[torch.isinf(times_m)]=0

        ### here means the resm_new as 1/v
        times_m = (i+1)*resm_new
        # print("times_m.shape",times_m.shape)
        # print("times_m",times_m)

        factors_m = factor_m * times_m
        # print("factors_m.shape",factor_m.shape)
        # print("factors_m",factor_m)
        if(i == 0):
            resm = factors_m
        else:
            resm = resm + factors_m*alpha
            alpha *= alpha
        # print("resm",resm)
        
        torch_mx = torch.mm(torch_mx,torch_mx)
    ## resm.requires_grad = False only can change on the leaf node
    return resm#mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo() #D^(-1/2)*A*D^(-1/2)^T

In [9]:
def get_top_k_matrix(A, k: int = 128):
    num_nodes = A.shape[0]
    row_idx = torch.arange(num_nodes)
    A[A.argsort(axis=0)[:num_nodes - k], row_idx] = 0.
    norm = A.sum(axis=0)
    norm[norm <= 0] = 1 # avoid dividing by zero
    return A/norm


def get_clipped_matrix(A, eps: float = 0.01):
    num_nodes = A.shape[0]
    A[A < eps] = 0.
    norm = A.sum(axis=0)
    norm[norm <= 0] = 1 # avoid dividing by zero
    return A/norm

def get_heat_matrix(
        adj_matrix,
        t: float = 5.0):
    num_nodes = adj_matrix.shape[0]
    A_tilde = adj_matrix + torch.eye(num_nodes)
    D_tilde = torch.diag(1/torch.sqrt(A_tilde.sum(axis=1)))
    H = D_tilde @ A_tilde @ D_tilde
    return torch.exp(-t * (torch.eye(num_nodes) - H))

"""
  t: 5.0
  k: 128
  eps: 0.0001
"""

'\n  t: 5.0\n  k: 128\n  eps: 0.0001\n'

In [10]:
def AF_matrix(adj, feature, hop_num):#D^(-1/2)^T*(AF)*(AF)^T*D^(-1/2)
    #L_AF self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    
    #PVM
    
    # adj 2708, feature 1433
    """Row-normalize sparse matrix"""
    """
    For adjancy matrix
    normalize_adj process
    adj--original matrix shape (2708, 2708)
    adj--degree sum matrix shape (2708, 1)
    adj--degree inv matrix shape (2708,)
    adj--degree no inf matrix shape (2708,)
    adj--degree norm diag matrix shape (2708, 2708)
    adj--res shape (2708, 2708)
    
    For feature matrix
    normalize process
    mx--original matrix shape (2708, 1433)
    mx--degree sum matrix shape (2708, 1)
    mx--degree inv matrix shape (2708,)
    mx--degree no inf matrix shape (2708,)
    mx--degree norm diag matrix shape (2708, 2708)
    mx--res shape (2708, 1433)
    """
    A_curhop = adj
    resm = adj
    dvec = torch.ones(adj.shape[0],1)#torch.ones(3*hop_num,1)
    L_AF = Parameter(torch.FloatTensor(1,3*hop_num))
    torch.nn.init.normal_(L_AF)
    """
    1 0 1
    1 0 1
    1 0 1
    对应一个对应AF矩阵的值
    """
    # PVM = np.array(torch.FloatTensor(3*hop_num,(adj.shape[1]+adj.shape[1]+feature.shape[1])*hop_num))
    PVM = torch.ones(3*hop_num,adj.shape[1])*L_AF[0][0]
    print("PVM.shape",PVM.shape)
    for i in range(1,3*hop_num):
        print("i'th PVM expansion")
        if((i%3) == 2) :
            PVM = torch.cat((PVM,torch.ones(3*hop_num,feature.shape[1])*L_AF[0][i]),1)
            print(PVM.shape)
        else:
            PVM = torch.cat((PVM,torch.ones(3*hop_num,adj.shape[1])*L_AF[0][i]),1)
            print(PVM.shape)
    """
    PVM = None
    for i in range(0,3*hop_num):
        if(i%3 == 0):
            PVM.concat((PVM,np.ones(3*hop_num,adj.shape[1])*L_AF[i]))
        else:
            PVM.concat((PVM,np.ones(3*hop_num,feature.shape[1])*L_AF[i]))
    """
    
    for i in range(0,hop_num):
        print("{}'th hop normalize_adj process".format(i))
        A_curhop_D = np.array(A_curhop.sum(1)).flatten() #每行求和，求得每个节点的出度列矩阵 A_curhop_D var mean
        ##print("A_curhop_D.shape:",A_curhop_D.shape)
        A_curhop_D = sp.diags(A_curhop_D) #var mean
        ##print("A_curhop_D.shape:",A_curhop_D.shape)
        # 进行A^k的k步传播，选择Feature矩阵的非0位置
        feature_curhop = A_curhop.dot(feature)#feature[A_cur_hop where not 0] #var mean
        ##print("feature_curhop.shape",feature_curhop.shape)
        #degree_hop = (A_cur_hop[A_cur_hop!=0]=A_curhop_D) #var mean
        """
        print("the adj matrix",A_curhop)
        print("the adj matrix.shape",A_curhop.shape)
        print("the degree matrix",A_curhop_D)
        print("the degree matrix.shape",A_curhop_D.shape)
        print("the adj feature matrix",feature_curhop) #A^hopnum * feature
        print("the adj feature matrix.shape",feature_curhop.shape)
        """
        """
        adj_rowsum = np.array(A_cur_hop.sum(1)) #每行求和，求得每个节点的出度列矩阵 A_curhop_D
        print("adj--degree sum matrix shape",adj_rowsum.shape)
        adj_r_inv_sqrt = np.power(adj_rowsum, -0.5).flatten()
        print("adj--degree inv matrix shape",adj_r_inv_sqrt.shape)#A_curhop_D^(-1/2)
        adj_r_inv_sqrt[np.isinf(adj_r_inv_sqrt)] = 0.
        print("adj--degree no inf matrix shape",adj_r_inv_sqrt.shape)
        adj_r_mat_inv_sqrt = sp.diags(adj_r_inv_sqrt) #A_D^(-1/2)
        print("adj--degree norm diag matrix shape",adj_r_mat_inv_sqrt.shape) #A_curhop_D^(-1/2)
        #norm_adj_resm = adj.dot(adj_r_mat_inv_sqrt).transpose().dot(adj_r_mat_inv_sqrt).tocoo()
        #print("adj--res shape",resm.shape)
        """
        """
        print("concatenate members shape")
        print("A_curhop.shape=",A_curhop.shape)
        print("A_curhop_D.shape",A_curhop_D.shape)
        """
        if(i !=0):
            resm = sp.hstack((A_curhop, resm))
        ##print("first time concat.shape",resm.shape)
        ##print("feature_curhop.shape",feature_curhop.shape)
        resm = sp.hstack((A_curhop_D, resm))
        resm = sp.hstack((feature_curhop, resm))
        ##print("the resm here is")
        ##print(resm.shape)
        A_curhop  =A_curhop.dot(adj)
    print("dvec shape")
    print(dvec.shape) #(9, 1)
    print("L_AF shape")
    print(L_AF.shape) # torch.Size([1, 9])
    print("PVM shape")
    print(PVM.shape) # (9, 20547) 1|0|0|1|1|0
    # (9,1)*(1,9)*(9,20547)
    """
    L_AF shape
    torch.Size([1, 9])
    PVM shape
    (9, 20547)
    | # of features : 1433
    | # of clases   : 7
    | # of train set : 140
    | # of val set   : 500
    | # of test set  : 1000
    """
    
    #resm #= sp.coo_matrix(a_matrix)#torch.from_numpy(resm) #tmp_coo=sp.coo_matrix(a_matrix)
    resm = resm
    values=resm.data
    indices=np.vstack((resm.row,resm.col))
    i=torch.LongTensor(indices)
    v=torch.LongTensor(values)
    resm =torch.sparse_coo_tensor(i,v,resm.shape)
    resm = resm.to_dense()
    
    mux_res = torch.mm(torch.mm(dvec,L_AF),PVM) #resm*L_AF*PVM
    print("mux_res result size",mux_res.shape)
    #print("preprocess matrix",mux_res)
    
    print("the actual A-F matrix shape",resm.shape)
    
    weighted_resm = torch.mul(mux_res,resm)#mux_res * resm
    
    mm_res = torch.mm(weighted_resm,weighted_resm.T)
    print("result multiplication shape",mm_res.shape)
    
    return mm_res #resm,L_AF,PVM #D^(-1/2)*A*D^(-1/2)^T

In [11]:
def train(epoch, model,record):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj, features)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward(retain_graph=True)  
    optimizer.step()
    model.eval()
    output = model(features, adj, features)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'acc_test: {:.4f}'.format(acc_test.item()),
          'time: {:.4f}s'.format(time.time() - t))
    record[acc_val.item()] = acc_test.item() #record: acc_val->acc_test

In [17]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, hop_num,bias=True):
        ## print("init_state of GCN")
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        ######此处定义了网路的parameter例如weight和bias 两者在init里面定义为parameter 在self保存为net的结构
        self.hop_num = hop_num
        self.L_AF = Parameter(torch.FloatTensor(1,3*hop_num))
        self.L_AF2 = Parameter(torch.FloatTensor(1,2*hop_num))
        factor = 1
        """
        for i in range(0,hop_num):
            self.L_AF[3*i] = factor
            self.L_AF[3*i+1] = factor*0.1
            self.L_AF[3*i+1] = factor*0.001
            factor *=0.1
        """
        self.weight = Parameter(torch.FloatTensor(in_features, out_features)) #线性Linear层负责convolutional变换 
        #是两个维度的转换 为1433,16 16.7
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features)) #bias大小为输出层上的偏执 可以为16 7
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        ## print("reset_state of GCN")
        ### reset linear layer
        stdv = 1. / math.sqrt(self.weight.size(1))
        ## print("stdv=",stdv)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj, feature):
        ###adj = self.AF_matrix(adj, feature, self.hop_num) #k-hop learnable AF hop
        #adj = normalize_adj_torch(adj) #1-hop normalization
        #adj = normalize_adj_torch_khop(adj,self.hop_num,0.1) #k-hop A prop with k-degree normalization

        # structure imbalance
        
        # AF-ture
        # structure sampling -> local sampling (walk) * global parameter

        # heat interaction in local domain --> global insight


        adj = self.AK_matrix(adj,self.hop_num) #k-hop A prop with learnable params
        """
        adj = get_heat_matrix(adj,t=5.0)
        adj = get_top_k_matrix(adj, k=128)
        adj = get_clipped_matrix(adj, eps=0.0001)
        """
        
        ##adj = normalize_diffusion_descent_multi_times(adj,3,0.1)
        adj = normalize_adj_torch_interaction_conservation(adj)
        """
        print("forward_state of GCN")
        print("input.shape=",input.shape) #input.shape= torch.Size([2708, 1433]) 0/1
        print("input=",input)
        print("adj.shape=",adj.shape) #adj.shape= torch.Size([2708, 2708]) float
        print("adj=",adj)
        """
        support = torch.mm(input, self.weight) #torch.mm矩阵乘法 线性层的个数为 1433*16 (in_features, out_features)
        """
        print("support.shape=",support.shape) #support.shape= torch.Size([2708, 16]) float
        print("support=",support)
        """
        output = torch.spmm(adj, support) #A*F*L spmm稀疏矩阵乘法
        """
        print("output.shape=",output.shape)#output.shape= torch.Size([2708, 16]) float
        print("output=",output)
        print("bias.shape=",self.bias.shape) #bias.shape= torch.Size([16]) float
        print("bias=",self.bias)
        """
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        print("repr_state of GCN")
        print(self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')')
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    def AFtune_matrix(self, adj, feature, hop_num):#D^(-1/2)^T*(AF)*(AF)^T*D^(-1/2) --somethinking feature-oriented tune and label-oriented tune
        # label
        # dense Feature
        adj = normalize_adj_torch(adj)
        A_curhop = adj
        resm = adj
        dvec = torch.ones(adj.shape[0],1)#torch.ones(3*hop_num,1)
        #L_AF = Parameter(torch.FloatTensor(1,3*hop_num))
        torch.nn.init.normal_(self.L_AF)
        # PVM = np.array(torch.FloatTensor(3*hop_num,(adj.shape[1]+adj.shape[1]+feature.shape[1])*hop_num))
        PVM = torch.ones(3*hop_num,adj.shape[1])*self.L_AF[0][0]
        factor = 1.0
        for i in range(1,3*hop_num):
            factor *= 0.1
            ## print("i'th PVM expansion")
            if((i%3) == 2) :
                PVM = torch.cat((PVM,torch.ones(3*hop_num,feature.shape[1])*self.L_AF[0][i]),1)
                ## print(PVM.shape)
            else:
                PVM = torch.cat((PVM,torch.ones(3*hop_num,adj.shape[1])*self.L_AF[0][i]),1)
                ## print(PVM.shape)

        for i in range(0,hop_num):
            ## print("{}'th hop normalize_adj process".format(i))
            A_curhop_D = A_curhop.sum(1).flatten() #每行求和，求得每个节点的出度列矩阵 A_curhop_D var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)
            A_curhop_D = A_curhop_D * torch.eye(A_curhop_D.shape[0]) #var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)torch.eye(A_curhop_D.shape[1])
            # 进行A^k的k步传播，选择Feature矩阵的非0位置
            feature_curhop = torch.mm(A_curhop,feature)
            if(i !=0):
                resm = torch.cat((A_curhop,resm), 1)
            resm = torch.cat((A_curhop_D, resm),1)
            #resm = sp.hstack((feature_curhop, resm))
            resm = torch.cat((feature_curhop, resm),1)
            A_curhop = torch.mm(A_curhop,adj)
            A_curhop = normalize_adj_torch(A_curhop)

        mux_res = torch.mm(torch.mm(dvec,self.L_AF),PVM) #resm*L_AF*PVM

        weighted_resm = torch.mul(mux_res,resm)#mux_res * resm

        mm_res = torch.mm(weighted_resm,weighted_resm.T)
        ## print("result multiplication shape",mm_res.shape)
        # mm_res.requires_grad=False

        return mm_res #resm,L_AF,PVM #D^(-1/2)*A*D^(-1/2)^T
    
    def AF_matrix(self, adj, feature, hop_num):#D^(-1/2)^T*(AF)*(AF)^T*D^(-1/2)
        #L_AF self.weight = Parameter(torch.FloatTensor(in_features, out_features))

        #PVM

        # adj 2708, feature 1433
        adj = normalize_adj_torch(adj)
        A_curhop = adj
        resm = adj
        dvec = torch.ones(adj.shape[0],1)#torch.ones(3*hop_num,1)
        #L_AF = Parameter(torch.FloatTensor(1,3*hop_num))
        torch.nn.init.normal_(self.L_AF)
        # PVM = np.array(torch.FloatTensor(3*hop_num,(adj.shape[1]+adj.shape[1]+feature.shape[1])*hop_num))
        PVM = torch.ones(3*hop_num,adj.shape[1])*self.L_AF[0][0]
        factor = 1.0
        ## print("PVM.shape",PVM.shape)
        for i in range(1,3*hop_num):
            factor *= 0.1
            ## print("i'th PVM expansion") A*F (A^n)*F
            if((i%3) == 2) :
                PVM = torch.cat((PVM,torch.ones(3*hop_num,feature.shape[1])*self.L_AF[0][i]*factor*0.00001),1)
                ## print(PVM.shape)
            else:
                PVM = torch.cat((PVM,torch.ones(3*hop_num,adj.shape[1])*self.L_AF[0][i]*factor),1)
                ## print(PVM.shape)

        for i in range(0,hop_num):
            ## print("{}'th hop normalize_adj process".format(i))
            A_curhop_D = A_curhop.sum(1).flatten() #每行求和，求得每个节点的出度列矩阵 A_curhop_D var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)
            A_curhop_D = A_curhop_D * torch.eye(A_curhop_D.shape[0]) #var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)torch.eye(A_curhop_D.shape[1])
            # 进行A^k的k步传播，选择Feature矩阵的非0位置
            feature_curhop = torch.mm(A_curhop,feature)
            if(i !=0):
                resm = torch.cat((A_curhop,resm), 1)
            ##print("first time concat.shape",resm.shape)
            ##print("feature_curhop.shape",feature_curhop.shape)
            #resm = sp.hstack((A_curhop_D, resm))
            resm = torch.cat((A_curhop_D, resm),1)
            #resm = sp.hstack((feature_curhop, resm))
            resm = torch.cat((feature_curhop, resm),1)
            A_curhop = torch.mm(A_curhop,adj)
            #A_curhop = normalize_adj_torch(A_curhop)->one-hop normalization multi-hop normalization

        mux_res = torch.mm(torch.mm(dvec,self.L_AF),PVM) #resm*L_AF*PVM

        weighted_resm = torch.mul(mux_res,resm)#mux_res * resm

        mm_res = torch.mm(weighted_resm,weighted_resm.T)
        ## print("result multiplication shape",mm_res.shape)
        # mm_res.requires_grad=False

        return mm_res #resm,L_AF,PVM #D^(-1/2)*A*D^(-1/2)^T

    def AK_matrix(self, adj, hop_num):#D^(-1/2)^T*(AF)*(AF)^T*D^(-1/2)
        #L_AF self.weight = Parameter(torch.FloatTensor(in_features, out_features))

        #PVM

        # adj 2708, feature 1433
        adj = normalize_adj_torch(adj)
        A_curhop = adj
        resm = adj
        dvec = torch.ones(adj.shape[0],1)#torch.ones(3*hop_num,1)
        #L_AF = Parameter(torch.FloatTensor(1,3*hop_num))
        torch.nn.init.normal_(self.L_AF2)
        # PVM = np.array(torch.FloatTensor(3*hop_num,(adj.shape[1]+adj.shape[1]+feature.shape[1])*hop_num))
        PVM = torch.ones(2*hop_num,adj.shape[1])*self.L_AF2[0][0]
        factor = 1.0
        ## print("PVM.shape",PVM.shape)
        for i in range(1,2*hop_num):
            factor *= 0.1
            ## print("i'th PVM expansion") A*F (A^n)*F
            PVM = torch.cat((PVM,torch.ones(2*hop_num,adj.shape[1])*self.L_AF2[0][i]*factor),1)
            ## print(PVM.shape)

        for i in range(0,hop_num):
            ## print("{}'th hop normalize_adj process".format(i))
            A_curhop_D = A_curhop.sum(1).flatten() #每行求和，求得每个节点的出度列矩阵 A_curhop_D var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)
            A_curhop_D = A_curhop_D * torch.eye(A_curhop_D.shape[0]) #var mean
            ##print("A_curhop_D.shape:",A_curhop_D.shape)torch.eye(A_curhop_D.shape[1])
            # 进行A^k的k步传播，选择Feature矩阵的非0位置
            # feature_curhop = torch.mm(A_curhop,feature)
            if(i !=0):
                resm = torch.cat((A_curhop,resm), 1)
            ##print("first time concat.shape",resm.shape)
            ##print("feature_curhop.shape",feature_curhop.shape)
            #resm = sp.hstack((A_curhop_D, resm))
            resm = torch.cat((A_curhop_D, resm),1)
            #resm = sp.hstack((feature_curhop, resm))
            # resm = torch.cat((feature_curhop, resm),1)
            A_curhop = torch.mm(A_curhop,adj)
            #A_curhop = normalize_adj_torch(A_curhop)->one-hop normalization multi-hop normalization

        mux_res = torch.mm(torch.mm(dvec,self.L_AF2),PVM) #resm*L_AF*PVM

        weighted_resm = torch.mul(mux_res,resm)#mux_res * resm

        mm_res = torch.mm(weighted_resm,weighted_resm.T)
        ## print("result multiplication shape",mm_res.shape)
        # mm_res.requires_grad=False

        return mm_res #resm,L_AF,PVM #D^(-1/2)*A*D^(-1/2)^T

In [18]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()


[STEP 1]: Upload cora dataset.


/tmp/ipykernel_641034/2773929969.py:94: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  objects.append(pkl.load(f, encoding='latin1'))


| # of nodes : 2708
| # of edges : 5278.0
NA.shape (2708, 1433)
NA.dtype float32
NA.type <class 'scipy.sparse._csr.csr_matrix'>
| # of features : 1433
| # of clases   : 7
| # of train set : 140
| # of val set   : 500
| # of test set  : 1000


/tmp/ipykernel_641034/2773929969.py:142: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


In [19]:
heat_matrix = get_heat_matrix(adj,t=5.0)
heat_matrix = get_top_k_matrix(heat_matrix, k=128)
heat_matrix = get_clipped_matrix(heat_matrix, eps=0.0001)

### 2 layers

In [20]:
# 正常2层的GCN
import torch.nn as nn
import torch.nn.functional as F


class GCN1(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout,hop_num):
        ## print("init_state of 2-layer GCN")
    
        super(GCN1, self).__init__() #inherent a GCN model
        
        
        #两层graph convolution
        self.gc1 = GraphConvolution(nfeat, nhid,bias=True,hop_num=3)
        self.gc2 = GraphConvolution(nhid, nclass,bias=True,hop_num=3)
        self.dropout = dropout
        self.hop_num = hop_num
        
        ## print("gc1=",self.gc1)
        ## print("gc2=",self.gc2)
        ## print("dropout=",self.dropout) #0.5

    def forward(self, x, adj,features):
        ## print("forward_state of 2-layer GCN")
        x_d = F.dropout(x, self.dropout, training=self.training) #对于feature进行drop_out 
        """
        print("x_d.shape=",x_d.shape) #x_d.shape= torch.Size([2708, 1433])
        print("x_d=",x_d)
        """
        """
        x_d= tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
        """
        ### layer 1 processed result
        x = self.gc1(x_d, adj, features) 
        #传入的参数有上一轮的一阶邻居聚合信息的映射 x_d, adj 
        #for new network here should be changed to AF^T*AF
        """
        print("layer1 res.shape",lrs1.shape) #layer1 res.shape torch.Size([2708, 16])
        print("layer1 res",lrs1) # AXW^0 #每次提取周围邻居的信息 A为边信息两边度的权重积 并aggregate到自己的信息来 对于1433维度信息做一个统一的线性层映射
        """
        ########################
        #我们现在的工作就是要把AX这个仅仅考虑结构的propagte加一个考虑feature的Propagate，
        #还有加一个异步的propagate，还有加一个考虑时间和距离衰减的propagate，同时在一定时间后给一定范围内的节点进行Normalization
        ########################
        x = F.relu(x)#self.gc1(x_d, adj, features)) ################## i-j 已用feature计算
        # X=relu(AXW^0) 这里考虑AXW^0 A[node_num,node_num]*X[node_num,feature_num]*W[feature_num,convolved_dim]
        # X=[node_num,pre_convolved_dim]
        # AXW^i=A[node_num,node_num]*X[node_num,pre_convolved_dim]*L[pre_convolved_dim,latter_convolved_dim]
        #A-F = [node_num,LongAF]
        # Org.A = (i,j)1-hopAttentioned-A
        # A-F^T*A-F = (i,j)multi-hopAttentioned-A
        """
        print("after relu GCN1 x.shape=",x.shape) #after relu GCN1 x.shape= torch.Size([2708, 16])
        print("after relu GCN1 x=",x)
        """
        """
        after relu GCN1 x= tensor([[0.0000, 0.0000, 0.1960,  ..., 0.1392, 0.0000, 0.0000],
        [0.0256, 0.0000, 0.1576,  ..., 0.2085, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1568,  ..., 0.1526, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0998,  ..., 0.1981, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1614,  ..., 0.1271, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1632,  ..., 0.1248, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
        """
        x = F.dropout(x, self.dropout, training=self.training)
        """
        print("after dropout x.shape=",x.shape)
        print("after dropout x=",x)
        """
        """
        after dropout x= tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3152,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3136,  ..., 0.3053, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.1997,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3229,  ..., 0.2542, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3264,  ..., 0.2497, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)
        """
        x = self.gc2(x, adj, features) #16->7 AXW^1
        """
        print("GCN2 x.shape=",x.shape) #GCN2 x.shape= torch.Size([2708, 7])
        print("GCN2 x=",x)
        """
        """
        GCN2 x= tensor([[-0.1629,  0.0549,  0.0964,  ...,  0.2143,  0.2979,  0.0673],
        [-0.1236,  0.0818,  0.0692,  ...,  0.2229,  0.3301,  0.0718],
        [-0.1413,  0.0453,  0.0838,  ...,  0.2303,  0.3301,  0.0501],
        ...,
        [-0.1111,  0.0988,  0.1083,  ...,  0.2313,  0.3120,  0.1059],
        [-0.0725,  0.1234,  0.0693,  ...,  0.2215,  0.3570,  0.1172],
        [-0.0930,  0.1279,  0.0991,  ...,  0.2040,  0.3516,  0.1457]],
       grad_fn=<AddBackward0>)
        """
        res = F.log_softmax(x, dim=1) #对于结果进行归一化 softmax(AXW^1)
        """
        print("res.shape=",res.shape) #res.shape= torch.Size([2708, 7])
        print("res=",res)
        """
        return F.log_softmax(x, dim=1)

In [21]:
model1 = GCN1(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5,hop_num=3) 
for item in model1.parameters():
        print(item.shape)

torch.Size([1, 9])
torch.Size([1, 6])
torch.Size([1433, 16])
torch.Size([16])
torch.Size([1, 9])
torch.Size([1, 6])
torch.Size([16, 7])
torch.Size([7])


In [22]:
# Model and optimizer
for i in range(2):
    print("parameters setting here")
    print('features.shape',features.shape) #features.shape torch.Size([2708, 1433])
    print("labels.max().item()",labels.max().item()) #labels.max().item() 6 [0,...,6]
    model1 = GCN1(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5,hop_num=3) 
    optimizer = optim.Adam(model1.parameters(),
                           lr=0.01, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(200):
        train(epoch,model1,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)
    
    """
    Optimization Finished!
    Total time elapsed: 28.3611s
    0.808 0.828
    0.806 0.83
    0.804 0.827
    0.802 0.834
    0.8 0.822
    0.798 0.825
    0.796 0.83
    0.794 0.826
    0.792 0.822
    0.79 0.828
    """

parameters setting here
features.shape torch.Size([2708, 1433])
labels.max().item() 6
Epoch: 0001 loss_train: 1.9481 acc_train: 0.1357 acc_val: 0.1640 acc_test: 0.1460 time: 1.9905s
Epoch: 0002 loss_train: 1.9384 acc_train: 0.1929 acc_val: 0.1640 acc_test: 0.1470 time: 1.9185s
Epoch: 0003 loss_train: 1.9444 acc_train: 0.1643 acc_val: 0.1640 acc_test: 0.1490 time: 1.8612s
Epoch: 0004 loss_train: 1.9485 acc_train: 0.1500 acc_val: 0.1620 acc_test: 0.1510 time: 1.8667s
Epoch: 0005 loss_train: 1.9389 acc_train: 0.1286 acc_val: 0.1680 acc_test: 0.1630 time: 1.8883s
Epoch: 0006 loss_train: 1.9327 acc_train: 0.1857 acc_val: 0.1820 acc_test: 0.2060 time: 1.9096s
Epoch: 0007 loss_train: 1.9331 acc_train: 0.1643 acc_val: 0.2260 acc_test: 0.2590 time: 1.8788s
Epoch: 0008 loss_train: 1.9218 acc_train: 0.1714 acc_val: 0.2960 acc_test: 0.3040 time: 1.9336s
Epoch: 0009 loss_train: 1.9302 acc_train: 0.2071 acc_val: 0.3240 acc_test: 0.3400 time: 1.8678s
Epoch: 0010 loss_train: 1.8853 acc_train: 0.3357 a

### 3层

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GCN1(nn.Module):
    def __init__(self, nfeat, nhid1,nhid2, nclass, dropout):
        super(GCN1, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1,bias=True)
        self.gc2 = GraphConvolution(nhid1, nhid2,bias=True)
        self.gc3 = GraphConvolution(nhid2, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj): 
#         x_d = F.dropout(x, self.dropout, training=self.training)
        
        x1 = F.relu(self.gc1(x, adj))
        x1_d = F.dropout(x1, self.dropout, training=self.training)
#         combined = torch.cat([feature, x1_d], dim=1)
        
        x2 = F.relu(self.gc2(x1_d, adj))
        x2_d = F.dropout(x2, self.dropout, training=self.training)
        
        x3 = self.gc3(x2_d, adj)
        return F.log_softmax(x3, dim=1)

In [ ]:
# Model and optimizer
for i in range(10):
    model1 = GCN1(nfeat=features.shape[1],
            nhid1=16,
            nhid2=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
    optimizer = optim.Adam(model1.parameters(),
                           lr=0.01, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(200):
        train(epoch,model1,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)

### 4层

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GCN1(nn.Module):
    def __init__(self, nfeat, nhid1,nhid2, nhid3,nclass, dropout):
        super(GCN1, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1,bias=True)
        self.gc2 = GraphConvolution(nhid1, nhid2,bias=True)
        self.gc3 = GraphConvolution(nhid2, nhid3,bias=True)
        self.gc4 = GraphConvolution(nhid3, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
#         x_d = F.dropout(x, self.dropout, training=self.training)
        
        x1 = F.relu(self.gc1(x, adj))
        x1_d = F.dropout(x1, self.dropout, training=self.training)
#         combined = torch.cat([feature, x1_d], dim=1)
        
        x2 = F.relu(self.gc2(x1_d, adj))
        x2_d = F.dropout(x2, self.dropout, training=self.training)
        
        x3 = F.relu(self.gc3(x2_d, adj))
        x3_d = F.dropout(x3, self.dropout, training=self.training)
        
        x4 = self.gc4(x3_d, adj)
        return F.log_softmax(x4, dim=1)

In [ ]:
# Model and optimizer
for i in range(10):
    model1 = GCN1(nfeat=features.shape[1],
            nhid1=16,
            nhid2=16,
            nhid3=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
    optimizer = optim.Adam(model1.parameters(),
                           lr=0.01, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(200):
        train(epoch,model1,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)

#### 5层

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GCN1(nn.Module):
    def __init__(self, nfeat, nhid1,nhid2, nhid3,nhid4,nclass, dropout):
        super(GCN1, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1,bias=True)
        self.gc2 = GraphConvolution(nhid1, nhid2,bias=True)
        self.gc3 = GraphConvolution(nhid2, nhid3,bias=True)
        self.gc4 = GraphConvolution(nhid3, nhid4,bias=True)
        self.gc5 = GraphConvolution(nhid4, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
#         x_d = F.dropout(x, self.dropout, training=self.training)
        
        x1 = F.relu(self.gc1(x, adj))
        x1_d = F.dropout(x1, self.dropout, training=self.training)
#         combined = torch.cat([feature, x1_d], dim=1)
        
        x2 = F.relu(self.gc2(x1_d, adj))
        x2_d = F.dropout(x2, self.dropout, training=self.training)
        
        x3 = F.relu(self.gc3(x2_d, adj))
        x3_d = F.dropout(x3, self.dropout, training=self.training)
        
        x4 = F.relu(self.gc4(x3_d, adj))
        x4_d = F.dropout(x4, self.dropout, training=self.training)
        
        x5 = self.gc5(x4_d, adj)
        return F.log_softmax(x5, dim=1)

In [ ]:
# Model and optimizer
for i in range(10):
    model1 = GCN1(nfeat=features.shape[1],
            nhid1=16,
            nhid2=16,
            nhid3=16,
            nhid4=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
    optimizer = optim.Adam(model1.parameters(),
                           lr=0.01, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(200):
        train(epoch,model1,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)

#### 6层

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GCN1(nn.Module):
    def __init__(self, nfeat, nhid1,nhid2, nhid3,nhid4,nhid5,nclass, dropout):
        super(GCN1, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1,bias=True)
        self.gc2 = GraphConvolution(nhid1, nhid2,bias=True)
        self.gc3 = GraphConvolution(nhid2, nhid3,bias=True)
        self.gc4 = GraphConvolution(nhid3, nhid4,bias=True)
        self.gc5 = GraphConvolution(nhid4, nhid5,bias=True)
        self.gc6 = GraphConvolution(nhid5, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
#         x_d = F.dropout(x, self.dropout, training=self.training)
        
        x1 = F.relu(self.gc1(x, adj))
        x1_d = F.dropout(x1, self.dropout, training=self.training)
#         combined = torch.cat([feature, x1_d], dim=1)
        
        x2 = F.relu(self.gc2(x1_d, adj))
        x2_d = F.dropout(x2, self.dropout, training=self.training)
        
        x3 = F.relu(self.gc3(x2_d, adj))
        x3_d = F.dropout(x3, self.dropout, training=self.training)
        
        x4 = F.relu(self.gc4(x3_d, adj))
        x4_d = F.dropout(x4, self.dropout, training=self.training)
        
        x5 = F.relu(self.gc5(x4_d, adj))
        x5_d = F.dropout(x5, self.dropout, training=self.training)
        
        x6 = self.gc6(x5_d, adj)
        return F.log_softmax(x6, dim=1)

In [ ]:
# Model and optimizer
for i in range(10):
    model1 = GCN1(nfeat=features.shape[1],
            nhid1=16,
            nhid2=16,
            nhid3=16,
            nhid4=16,
            nhid5=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
    optimizer = optim.Adam(model1.parameters(),
                           lr=0.01, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(200):
        train(epoch,model1,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)